<a href="https://colab.research.google.com/github/LordLean/Acquired-Intelligence-Adaptive-Behaviour/blob/master/AIAB_Labs/microbialGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib as plt
import random

In [0]:
# (benefit, cost)
knapsack = [(5,3),(6,2),(1,4),(9,5),(2,8),(8,9),(4,10),(3,1),(7,6),(10,7)]
crossover = {"One Point":0, "Multi Point":1, "Uniform":2}
config = {"kp":knapsack, "capacity":20}

In [0]:
class MicrobialGA(object):


  def __init__(self, kp, capacity, crossover_op=0, population_size=100, tournaments=100, mutation_rate=0.3):
    self.kp = kp # Knapsack to be evaluated.
    self.capacity = capacity # Knapsack capacity.
    self.crossover_op = crossover_op # Set desired crossover method.
    self.population_size = population_size # Population size of genotypes.
    self.tournaments = tournaments # Tournaments to run for. 
    self.mutation_rate = mutation_rate # Mutation rate to effect rate of evolution.
    self.geno_shape = len(self.kp) # Shape of desired genotype.
    self.population = np.zeros(shape=(population_size, self.geno_shape), dtype=int) # Hold all genotypes/solutions.


  def initalize(self):
    # Initialze each solution in population to random binary values.
    for i, genotype in enumerate(self.population):
      current_sum = self.capacity + 1
      # Ensure random values do not cause a cost above capacity.
      while current_sum > self.capacity:
        for i, _ in enumerate(genotype):
          genotype[i] = random.randint(0,1)
          # Update current sum until it holds a value less than capacity.
          current_sum = np.sum([self.kp[i][1] * genotype[i] for i, _ in enumerate(self.kp)])


  # NOT VERY SCALABLE IS IT!
  def selection(self):
    # Create indicies to compare without out of bounds exception.
    upper_bound = self.geno_shape - 1
    index_one = random.randint(0,upper_bound)
    index_two = int()
    p = random.uniform(0,1)
    # If p < 1/4 match genotype with previous neighbour at distance 2. 
    if p < 0.25:
      index_two = index_one - 2
    # If 1/4 <= p < 1/2 match genotype with previous neighbour at distance 1. 
    elif p < 0.5:
      index_two = index_one - 1
    # If 1/2 <= p < 3/4 match genotype with next neighbour at distance 1. 
    elif p < 0.75:
      # Try-catch to avoid out of bounds exception. If index does not exist set to 0.
      try:
        index_two = index_one + 1
        # Try catch was maybe not the best solution as had to force a possible error but this was the imp. that I had started.
        geno_one = self.population[index_one] 
        geno_two = self.population[index_two]
        return geno_one, geno_two
      except:
        index_two = 0
    # If 3/4 <= p <= 1 match genotype with next neighbour at distance 2. 
    else:
      # Try-catch to avoid out of bounds exception. If index does not exist set to 1.
      try:
        index_two = index_one + 2
        geno_one = self.population[index_one] 
        geno_two = self.population[index_two]
        return geno_one, geno_two
      except:
        index_two = 1
    # State the two "neighbouring" genotypes to return.
    geno_one = self.population[index_one] 
    geno_two = self.population[index_two] 
    return geno_one, geno_two


  def fitness(self, geno_one, geno_two):
    # Fitness returns the better of two genotypes in the form: Winner, loser.
    # Benefit-cost evaluation.
    geno_one_bene = np.sum([self.kp[i][0] * geno_one[i] for i, _ in enumerate(self.kp)])
    geno_one_cost = np.sum([self.kp[i][1] * geno_one[i] for i, _ in enumerate(self.kp)])
    #Benefit-cost evaluation.
    geno_two_bene = np.sum([self.kp[i][0] * geno_two[i] for i, _ in enumerate(self.kp)])
    geno_two_cost = np.sum([self.kp[i][1] * geno_two[i] for i, _ in enumerate(self.kp)])
    # Case: benefits are equal, cost is evaluated.
    if geno_one_bene == geno_two_bene:
      if geno_one_cost <= geno_two_cost:
        return geno_one, geno_two
      else:
        return geno_two, geno_one
    # If genotype one has a greater benefit then that genotype is returned.
    elif geno_one_bene > geno_two_bene:
      return geno_one, geno_two
    # If genotype one has a lesser benefit then that genotype is returned.
    else:
      return geno_two, geno_one


  def mutate(self, g):
    # Declare variable to hold mutated genotype
    mutated_genotype = np.zeros(self.geno_shape, dtype=int)
    # Setup: while loop.
    current_sum = self.capacity + 1
    # Values will be reassigned until the mutated genotype cost is below capacity.
    while current_sum > self.capacity:
      # Copy genotype to temporary genotype.
      mutated_genotype = np.copy(g)
      for i, _ in enumerate(mutated_genotype):
        num = random.uniform(0,1)  
        # Allow mutation rate to reassign values.
        if num < self.mutation_rate:
          # If "on" switch off.
          if mutated_genotype[i]:
            mutated_genotype[i] = 0
          # If "off" switch on.
          else:
            mutated_genotype[i] = 1
      # Update current sum for the while loop to evaluate.
      current_sum =  np.sum([self.kp[i][1] * mutated_genotype[i] for i, _ in enumerate(self.kp)])
    g = mutated_genotype
    # Return mutated genotype to be used for comparison.
    return g


  #### COULD ADD FUNCTIONALITY TO PREVENT MUTATION AT CROSSED OVER POINTS??
  def crossover(self):
    # One Point Crossover.
    if self.crossover_op == 0:
      geno_one, geno_two = self.selection()
      # Winner, Loser
      geno_one, geno_two = self.fitness(geno_one, geno_two)
      # Find index to split winning genotype portion for crossover.
      upper_bound = self.geno_shape
      cross_index = random.randint(-upper_bound, upper_bound)
      if cross_index <= 0:
        geno_two[:cross_index] = geno_one[:cross_index]
      else:
        geno_two[cross_index:] = geno_one[cross_index:]
      # Mutate the crossed over loser genotype.
      geno_two = self.mutate(geno_two)


In [0]:
mga = MicrobialGA(config["kp"], config["capacity"], crossover["One Point"], population_size=10)
mga.initalize()
one, two = mga.selection()
one, two = mga.fitness(one,two)
mga.crossover()

In [176]:
def one_point_cross_test(num,first=None,second=None):
  if first:
    one = np.zeros(10, dtype=int)
    two = np.zeros(10, dtype=int)
    one[:] = 3
    two[:num] = one[:num]
    print(one)
    print(two)
  
  elif second:
    one = np.zeros(10, dtype=int)
    two = np.zeros(10, dtype=int)
    one[:] = 3
    two[num:] = one[num:]
    print(one)
    print(two)

one_point_cross_test(-6,first=True,second=True)

[3 3 3 3 3 3 3 3 3 3]
[3 3 3 3 0 0 0 0 0 0]


In [77]:
storage = list()
for _ in range(100):
  num = random.uniform(0,10)
  num = np.round(num).astype("int")
  num1 = random.randint(-1,1)
  storage.append(num1)

max(storage)

print(storage)

[-1, -1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, -1, 1, 1, -1, 0, 1, -1, 1, 1, -1, 1, 1, -1, 0, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, -1, 1, 0, 0, 1, -1, -1, -1, 0, 1, -1, 0, 0, 1, 1, 1, 1, -1, 0, 1, 1, 1, 1, 1, 0, 1, 1, -1, 1, -1, 1, 0, 1, 0, 1, 1, 0, 1, -1, 0, 1, 1, 0, 0, -1, 0]


In [54]:
one, two = mga.selection()
ONE = np.sum([config["kp"][i][0] * one[i] for i, _ in enumerate(config["kp"])])
TWO = np.sum([config["kp"][i][0] * two[i] for i, _ in enumerate(config["kp"])])
print("Array One: {} and Array Two: {}".format(one,two))
print("Array One: {} and Array Two: {}".format(ONE,TWO))
one, two = mga.fitness(one,two)
print("Array One: {} and Array Two: {}".format(one,two))

Array One: [0 0 1 0 1 0 0 1 0 0] and Array Two: [1 0 0 0 1 0 0 0 0 1]
Array One: 6 and Array Two: 17
Array One: [1 0 0 0 1 0 0 0 0 1] and Array Two: [0 0 1 0 1 0 0 1 0 0]


In [25]:
mga = MicrobialGA(config["kp"], config["capacity"], 10)
mga.population
for i, item in enumerate(mga.population):
  current_sum = 20 + 1
  # Ensure random values do not cause a cost above capacity.
  while current_sum > 20:
    for i, _ in enumerate(item):
      num = random.uniform(0,1)  
      item[i] = np.round(num)
      # Update current sum until it holds a value less than capacity.
      current_sum =  np.sum([mga.kp[i][1] * item[i] for i, _ in enumerate(mga.kp)])
mga.population

array([[0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 1],
       [0, 1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [1, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
       [1, 1, 0, 1, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 0, 0,

In [36]:
print(mga.population)
print()
print(mga.population[0])
print(mga.population[-1])

[[1 1 0 0 0 0 1 0 0 0]
 [0 1 1 0 0 0 0 1 0 0]
 [0 0 1 1 0 0 0 1 0 0]
 [0 1 1 0 0 1 0 0 0 0]
 [0 1 0 1 0 0 0 1 0 1]
 [1 0 0 0 0 1 0 1 0 0]
 [0 1 0 0 1 0 0 1 0 1]
 [0 0 0 0 0 1 0 1 0 0]
 [1 1 0 0 1 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 1 0]]

[1 1 0 0 0 0 1 0 0 0]
[0 0 1 0 0 0 0 0 1 0]


In [131]:
test = np.zeros(10,dtype=int)
upper_bound = 10 - 1
num = random.uniform(0,upper_bound)  
num = np.round(num).astype("int")
num = num + 1
test[0] = 999
try:
  print(test[num])
except:
  print(num)
  print(test[0])

9.0
999
